In [ ]:
import numpy as np
import networkx as nx
from bokeh.io import output_notebook
from bokeh.io import output_file, show, save
from bokeh.plotting import figure, from_networkx
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import collections
from itertools import combinations

In [ ]:
def parseData(filepath):
    data = pd.read_csv(filepath)
    graph = nx.Graph()

    nodes = data['username'].unique()

    for node in nodes:
        graph.add_node(node)

    grouped_by_page_and_thread = data.groupby(['page_name', 'thread_subject'])

    for page_and_thread, dataframe in grouped_by_page_and_thread:
        users = dataframe['username'].unique()
        for i in range(len(users)):
            for j in range(i + 1, len(users)):
                graph.add_edge(users[i], users[j])

    return graph
        





### Load Graphs

In [ ]:
if __name__ == "__main__":
    filepath = 'datasets/PROPERTIES_FOR_DELETION_SML.csv'
    filepath2 = "datasets/WIKIPROJECTS_MED.csv"
    filepath3 = "datasets/USERS_LRG.csv"

    graph1 = parseData(filepath=filepath)
    graph2 = parseData(filepath=filepath2)
    graph3 = parseData(filepath=filepath3)

In [ ]:
nx.draw(graph1, with_labels = False)
plt.show()

In [ ]:
nx.draw(graph2, with_labels = False)
plt.show()

In [ ]:
nx.draw(graph3, with_labels = False)
plt.show()

In [ ]:
 # If we want to plot the graph = subgraph(largest_component)
def bokeh_plot_simple(graph:nx.Graph, title:str, scale=2, crop_factors = None):
    
    crop_factors = dict(x_range=(-1.1,1.1), y_range=(-1.1,1.1)) \
        if crop_factors is None else crop_factors

    plot = figure(
        title=title, tools="",
        toolbar_location=None, **crop_factors)

    mapping = dict((n, i) for i, n in enumerate(graph.nodes))
    graph_mapped = nx.relabel_nodes(graph, mapping)

    graph_plot = from_networkx(
        graph_mapped, nx.spring_layout, scale=scale, center=(0,0))
    plot.renderers.append(graph_plot)

    #output_file("networkx_graph.html")
    show(plot)

output_notebook()

In [ ]:
bokeh_plot_simple(graph1, 'Graph 1', 4)

In [ ]:
bokeh_plot_simple(graph2, 'Graph 2', 1)

In [ ]:
bokeh_plot_simple(graph3, 'Graph 3', 1)

## i) Characteristics

### 1) graph statistics

In [ ]:
def get_nodes_with_highest_degree(graph):
    degree_sequence = [d for n, d in graph.degree()]

    max_degree = max(degree_sequence)
    nodes_with_max_degree = [n for n, d in graph.degree() if d == max_degree]

    return(max_degree, nodes_with_max_degree)

In [ ]:
# i)1) Graph Statistics
def print_graph_statistics(graph:nx.Graph):
    print("Number of nodes: {}\nNumber of edges: {}".format(
        graph.number_of_nodes(), graph.number_of_edges()
    ))
    print("Number of connected components: {}".format(
        nx.algorithms.components.number_connected_components(graph),
    ))
    print("Average degree: {}\nClustering coefficient: {}".format(
        np.mean([deg for _, deg in graph.degree]),
        nx.algorithms.cluster.average_clustering(graph)
    ))
    max_degree, nodes_with_max_degree = get_nodes_with_highest_degree(graph)
    print(f'Max degree: {max_degree}\nNodes with max degree:{nodes_with_max_degree}')

    try:  # attempt to compute the diameter of the graph
        diam = nx.algorithms.approximation.distance_measures.diameter(graph)
        print("Graph diameter: {}".format(diam))
    except:  # an error has  occurred
        print("\nERROR: Could not compute the diameter of the graph.")

In [ ]:
print_graph_statistics(graph1)

In [ ]:
print_graph_statistics(graph2)

In [ ]:
print_graph_statistics(graph3)

In [ ]:
def get_degree_count_distribution(graph):
    degree_sequence = [d for n, d in graph.degree()]
    degree_count = collections.Counter(degree_sequence)
    deg, cnt = zip(*degree_count.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.8)
    plt.title('Degree Histogram')
    plt.ylabel('Count')
    plt.xlabel('Degree')
    plt.show


In [ ]:
get_degree_count_distribution(graph1)

In [ ]:
get_degree_count_distribution(graph2)

In [ ]:
get_degree_count_distribution(graph3)

### 2) high level stats

In [ ]:
def print_connected_statistics_with_average_shortest_path(component:nx.Graph):
    print("Number of nodes: {}\nNumber of edges: {}".format(
        component.number_of_nodes(), component.number_of_edges()
    ))
    print("Average path length: {}".format(
    nx.average_shortest_path_length(component)
    ))
    print("Number of connected components: {}".format(
        nx.algorithms.components.number_connected_components(component),
    ))
    print("Average degree: {}\nClustering coefficient: {}".format(
        np.mean([deg for _, deg in component.degree]),
        nx.algorithms.cluster.average_clustering(component)
    ))
    max_degree, nodes_with_max_degree = get_nodes_with_highest_degree(graph)
    print(f'Max degree: {max_degree}\nNodes with max degree:{nodes_with_max_degree}')


    try:  # attempt to compute the diameter of the graph
        diam = nx.algorithms.approximation.distance_measures.diameter(component)
        print("Graph diameter: {}".format(diam))
    except:  # an error has  occurred
        print("\nERROR: Could not compute the diameter of the graph.")

In [ ]:
def print_statistics_for_largest_component(graph:nx.Graph):
    largest_component = max(nx.connected_components(graph), key=len)
    graph_largest_components = graph.subgraph(largest_component)
    print_connected_statistics_with_average_shortest_path(graph_largest_components)
    #Could do for all components
    # for i, conn_component in enumerate(
    #     nx.connected_components(graph)):
    #     print(f"[Graph component {i}]")
    #     sub_graph = graph.subgraph(conn_component)  # XXX Careful to manupulations!
    #     print_connected_component_statistics(sub_graph)
    #     print("-"*50 + "\n")

In [ ]:
print_statistics_for_largest_component(graph1)

In [ ]:
print_statistics_for_largest_component(graph2)

In [ ]:
print_statistics_for_largest_component(graph3)

In [ ]:
def get_degree_count_distribution_largest_component(graph):
    degree_sequence = [d for n, d in graph.degree()]
    degree_count = collections.Counter(degree_sequence)
    deg, cnt = zip(*degree_count.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.8)
    plt.title('Degree Histogram')
    plt.ylabel('Count')
    plt.xlabel('Degree')
    plt.show

In [ ]:
get_degree_count_distribution_largest_component(graph1)

In [ ]:
get_degree_count_distribution_largest_component(graph2)

In [ ]:
get_degree_count_distribution_largest_component(graph3)

### 3) Node level statistics

In [ ]:
def get_node_level_descriptors(graph:nx.Graph):
    degrees = [d for _, d in graph.degree()]
    degree_centrality = [d for _, d in nx.degree_centrality(graph).items()]
    ccoeffs = [d for _, d in nx.algorithms.cluster.clustering(graph).items()]
    ccentra = [d for _, d in nx.closeness_centrality(graph).items()]

    return {'degrees': degrees, 'degree_centrality': degree_centrality, 'clustering coefficients': ccoeffs, 'closenes centrality': ccentra}

In [ ]:
def plot_helper_node_level_descriptors(descriptors, titles, key):
    data = {titles[i]: descriptors[i][key] for i in range(len(titles))}
    sns.displot(data, height=4, aspect=2, kde=True)


def show_node_level_descriptors_degrees(graphs=[graph1, graph2, graph3], titles=['graph1', 'graph2', 'graph3']):
    
    descriptors = [get_node_level_descriptors(graph) for graph in graphs]

    print('Degrees')
    plot_helper_node_level_descriptors(descriptors, titles, 'degrees')

def show_node_level_descriptors_clustering_centrality(graphs=[graph1, graph2, graph3], titles=['graph1', 'graph2', 'graph3']):
    
    descriptors = [get_node_level_descriptors(graph) for graph in graphs]

    print('Clustering Coefficient')
    plot_helper_node_level_descriptors(descriptors, titles, 'clustering coefficients')

def show_node_level_descriptors_closeness_centrality(graphs=[graph1, graph2, graph3], titles=['graph1', 'graph2', 'graph3']):
    
    descriptors = [get_node_level_descriptors(graph) for graph in graphs]

    print('Closeness Centrality')
    plot_helper_node_level_descriptors(descriptors, titles, 'closenes centrality')

In [ ]:

show_node_level_descriptors_degrees()

In [ ]:
show_node_level_descriptors_clustering_centrality()

In [ ]:
show_node_level_descriptors_closeness_centrality()

## ii) Shortest Paths

In [ ]:
# Djikstra
def get_shortest_path_largest_component_Dijkstra(graph:nx.Graph, farthest_nodes:tuple):
    start_node, end_node = farthest_nodes

    spath = nx.algorithms.dijkstra_path(graph, start_node, end_node)
    print("\nShortest path: " + " -> ".join([str(n) for n in spath]))

    print("How long is the path among these farthest nodes? {}".format(
    len(spath) - 1))  # here we do -1 to avoid counting the starting node!
    print(f'Should be the same as the diameter of the graph!!!: {nx.algorithms.approximation.distance_measures.diameter(graph)}')

In [ ]:
# BF
def get_shortest_path_largest_component_BF(graph:nx.Graph, farthest_nodes):
    start_node, end_node = farthest_nodes

    print(f"Start node: {start_node}\nEnd node: {end_node}") 

    spath = nx.algorithms.bellman_ford_path(graph, start_node, end_node)
    print("\nShortest path: " + " -> ".join([str(n) for n in spath]))

    print("How long is the path among these farthest nodes? {}".format(
    len(spath) - 1))  # here we do -1 to avoid counting the starting node!
    print(f'Should be the same as the diameter of the graph!!!: {nx.algorithms.approximation.distance_measures.diameter(graph)}')

In [ ]:
graph1_largest_component = graph1.subgraph(max(nx.connected_components(graph1), key=len))

graph2_largest_component = graph2.subgraph(max(nx.connected_components(graph2), key=len))

graph3_largest_component = graph3.subgraph(max(nx.connected_components(graph3), key=len))


In [ ]:
import sys
import itertools

In [ ]:
spinner_chars = itertools.cycle('|/-\\')  # Create a cycle iterator for the spinner


def update_spinner(spinner_chars, percent):
    sys.stdout.write('\r' + 'Loading: ' + next(spinner_chars) + '   ' + str(round(percent, 2)) + '% ')
    sys.stdout.flush()



In [ ]:
def get_farthest_nodes(graph):
    max_length = 0
    farthest_nodes = None
    graph_diam = nx.algorithms.approximation.distance_measures.diameter(graph)
    size = len(graph.nodes())
    size *= len(graph.nodes())
    iter = 0

    for possible_start_node in graph.nodes():
        update_spinner(spinner_chars, (iter*100/size))
        for possible_end_node in graph.nodes():
            if possible_start_node != possible_end_node:
                iter += 1
                try:
                    update_spinner(spinner_chars, (iter*100/size))
                    length = nx.shortest_path_length(graph, source = possible_start_node, target=possible_end_node, method='dijkstra')
                    if length > max_length:
                        max_length = length
                        farthest_nodes = (possible_start_node, possible_end_node)
                    elif max_length == graph_diam:
                        update_spinner(spinner_chars, (iter*100/size))
                        return farthest_nodes
                except nx.NetworkXNoPath:
                    continue
                    

    return farthest_nodes

In [ ]:
# leveraging the property of the diameter's endpoints being part of the longest shortest path from any node
def get_farthest_nodes_efficient(graph:nx.Graph):
    node = list(graph.nodes())[0]

    distances_from_node = nx.single_source_shortest_path_length(graph, node)
    start_node = max(distances_from_node, key=distances_from_node.get)

    distances_from_start_node = nx.single_source_shortest_path_length(graph, start_node)
    end_node = max(distances_from_start_node, key=distances_from_start_node.get)

    return(start_node, end_node)

In [ ]:
def check_diam_is_in_component(graph:nx.Graph, largest_component:nx.Graph):
    try:
        largest_comp_diam = nx.algorithms.approximation.distance_measures.diameter(largest_component)
    except:
        print('This component has no diameter')
        return False

    connected_components = [list(component) for component in nx.connected_components(graph)]
    for comp in connected_components:
        try:  # attempt to compute the diameter of the graph
            diam = nx.algorithms.approximation.distance_measures.diameter(graph)
            if nx.algorithms.approximation.distance_measures.diameter(comp) > largest_comp_diam:
                return False
        except:  # an error has  occurred
            continue
        
    return True



In [ ]:
# Check to see if largest path is in largest component
print('Graph 1:')
print(check_diam_is_in_component(graph1, graph1_largest_component))
print('\nGraph 2: ')
print(check_diam_is_in_component(graph2, graph2_largest_component))
print('\nGraph 3:')
print(check_diam_is_in_component(graph3, graph3_largest_component))

In [ ]:
graph1_farthest_nodes = get_farthest_nodes_efficient(graph1_largest_component) # diam = 4

In [ ]:
graph2_farthest_nodes  = get_farthest_nodes_efficient(graph2_largest_component) # diam = 8

In [ ]:
# we have assumed that the largest path will be in the largest component
graph3_farthest_nodes = get_farthest_nodes_efficient(graph3_largest_component) # diam = 10

In [ ]:
get_shortest_path_largest_component_Dijkstra(graph1_largest_component, graph1_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_Dijkstra(graph2_largest_component, graph2_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_Dijkstra(graph3_largest_component, graph3_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(graph1_largest_component, graph1_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(graph2_largest_component, graph2_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(graph3_largest_component, graph3_farthest_nodes)

## iii) Where is it on random <-> small world <-> regular

In [ ]:
def get_equivalent_random_graph(graph:nx.Graph, draw=True):
    # n : number of nodes
    # p : frequency of edge occurence
        # max edges: n (n - 1) / 2
        # frequency of edge occurence: number of edges / max edges
    n = graph.number_of_nodes()
    number_edges = graph.number_of_edges()
    max_edges = n*(n-1)/2
    p = number_edges/max_edges
    equivalen_random =  nx.erdos_renyi_graph(n=n, p=p)
    
    nx.draw(equivalen_random, with_labels=False) if draw else 0
    return equivalen_random



In [ ]:
def get_equivalent_regular_graph(graph:nx.Graph, draw=True):
    regular_graph = nx.Graph()

    nodes = graph.number_of_nodes()

    regular_graph.add_nodes_from(list(range(nodes)))

    #list_nodes = list(graph.nodes())
    for node in regular_graph:
        next_one = (node + 1) % nodes 
        jump_node = (node + 2) % nodes
        regular_graph.add_edge(node, next_one)
        regular_graph.add_edge(node, jump_node)

    if draw:
        fig, ax = plt.subplots(figsize=(10,10))
        nx.draw(regular_graph, pos=nx.circular_layout(regular_graph), with_labels=False)
    return regular_graph

In [ ]:
def print_graph_comparisons_statistics(graph:nx.Graph):
    equivalent_random = get_equivalent_random_graph(graph)
    print_graph_statistics(equivalent_random)
    equivalent_regular = get_equivalent_regular_graph(graph)
    print_graph_statistics(equivalent_regular)

In [ ]:
graph1_equivalent_random = get_equivalent_random_graph(graph1, False)
graph2_equivalent_random = get_equivalent_random_graph(graph2, False)
graph3_equivalent_random = get_equivalent_random_graph(graph3, False)

graph1_equivalent_regular = get_equivalent_regular_graph(graph1, False)
graph2_equivalent_regular = get_equivalent_regular_graph(graph2, False)
graph3_equivalent_regular = get_equivalent_regular_graph(graph3, False)

In [ ]:
print_graph_statistics(graph1_equivalent_random)
print('\n')
print_statistics_for_largest_component(graph1_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(graph1_equivalent_regular)

In [ ]:
print_graph_statistics(graph2_equivalent_random)
print('\n')
print_statistics_for_largest_component(graph2_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(graph2_equivalent_regular)

In [ ]:
print_graph_statistics(graph3_equivalent_random)
print('\n')
print_statistics_for_largest_component(graph3_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(graph3_equivalent_regular)

In [ ]:
get_degree_count_distribution(graph1_equivalent_random)

In [ ]:
get_degree_count_distribution(graph1_equivalent_regular)

In [ ]:
get_degree_count_distribution(graph2_equivalent_random)

In [ ]:
get_degree_count_distribution(graph2_equivalent_regular)

In [ ]:
get_degree_count_distribution(graph3_equivalent_random)

In [ ]:
get_degree_count_distribution(graph3_equivalent_regular)

In [ ]:

def print_node_level_comparison(graph:nx.Graph, equivalent_random:nx.Graph, equivalent_regular:nx.Graph):

    show_node_level_descriptors_degrees([graph, equivalent_random, equivalent_regular], ['graph', 'random', 'regular'])
    print('\n'*4)
    show_node_level_descriptors_clustering_centrality([graph, equivalent_random, equivalent_regular], ['graph', 'random', 'regular'])
    print('\n'*4)
    show_node_level_descriptors_closeness_centrality([graph, equivalent_random, equivalent_regular], ['graph', 'random', 'regular'])


In [ ]:
print_node_level_comparison(graph1, graph1_equivalent_random, graph1_equivalent_regular)

In [ ]:
print_node_level_comparison(graph2, graph2_equivalent_random, graph2_equivalent_regular)

In [ ]:
print_node_level_comparison(graph3, graph3_equivalent_random, graph3_equivalent_regular)

## v) Two editors are connected iff they have both contributed to any thread in the same page, but not necessarily to the same thread? ( I.e. we would have more connections in the network)

In [ ]:
def create_graph_connected_by_thread_in_same_page(network_data:str):
    net = pd.read_csv(network_data)

    graph = nx.Graph()

    for page, dataframe in net.groupby('page_name'):
        nodes = dataframe['username'].unique()
        for i in range(len(nodes)):
            for j in range (i + 1, len(nodes)):
                graph.add_edge(nodes[i], nodes[j])

    return graph



#### - statistics
#### - statistics of largest comp
#### - node level stats
#### - shortest paths (largest comp)
#### - where on the regular <-> random network

In [ ]:
net1 = create_graph_connected_by_thread_in_same_page('datasets/PROPERTIES_FOR_DELETION_SML.csv')
net2 = create_graph_connected_by_thread_in_same_page("datasets/WIKIPROJECTS_MED.csv")
net3 = create_graph_connected_by_thread_in_same_page("datasets/USERS_LRG.csv")

In [ ]:
nx.draw(net1, with_labels = False)
plt.show()

In [ ]:
nx.draw(net2, with_labels = False)
plt.show()

In [ ]:
nx.draw(net3, with_labels = False)
plt.show()

In [ ]:
bokeh_plot_simple(net1, 'Net 1', 4)

In [ ]:
bokeh_plot_simple(net2, 'Net 2', 1)

In [ ]:
bokeh_plot_simple(net3, 'Net 3', 2)

In [ ]:
# Statistics
print_graph_statistics(net1)

In [ ]:
print_graph_statistics(net2)

In [ ]:
print_graph_statistics(net3)

In [ ]:
# Statistics largest comp
print_statistics_for_largest_component(net1)

In [ ]:
print_statistics_for_largest_component(net2)

In [ ]:
print_statistics_for_largest_component(net3)

In [ ]:
# node level stats
show_node_level_descriptors_degrees([net1, net2, net3], ['net1', 'net2', 'net3'])

In [ ]:
show_node_level_descriptors_clustering_centrality([net1, net2, net3], ['net1', 'net2', 'net3'])

In [ ]:
show_node_level_descriptors_closeness_centrality([net1, net2, net3], ['net1', 'net2', 'net3'])

In [ ]:
# shortest paths

net1_largest_component = net1.subgraph(max(nx.connected_components(net1), key=len))

net2_largest_component = net2.subgraph(max(nx.connected_components(net2), key=len))

net3_largest_component = net3.subgraph(max(nx.connected_components(net3), key=len))

In [ ]:
# Check to see if largest path is in largest component
print('Net 1:')
print(check_diam_is_in_component(net1, net1_largest_component))
print('\nNet 2: ')
print(check_diam_is_in_component(net2, net2_largest_component))
print('\nNet 3:')
print(check_diam_is_in_component(net3, net3_largest_component))

In [ ]:
net1_farthest_nodes  = get_farthest_nodes_efficient(net1_largest_component)

In [ ]:
net2_farthest_nodes  = get_farthest_nodes_efficient(net2_largest_component)

In [ ]:
net3_farthest_nodes  = get_farthest_nodes_efficient(net3_largest_component)

In [ ]:
get_shortest_path_largest_component_Dijkstra(net1_largest_component, net1_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_Dijkstra(net2_largest_component, net2_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_Dijkstra(net3_largest_component, net3_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(net1_largest_component, net1_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(net2_largest_component, net2_farthest_nodes)

In [ ]:
get_shortest_path_largest_component_BF(net3_largest_component, net3_farthest_nodes)

In [ ]:
net1_equivalent_random = get_equivalent_random_graph(net1, False)
net2_equivalent_random = get_equivalent_random_graph(net2, False)
net3_equivalent_random = get_equivalent_random_graph(net3, False)

net1_equivalent_regular = get_equivalent_regular_graph(net1, False)
net2_equivalent_regular = get_equivalent_regular_graph(net2, False)
net3_equivalent_regular = get_equivalent_regular_graph(net3, False)

In [ ]:
print_graph_statistics(net1_equivalent_random)
print('\n')
print_statistics_for_largest_component(net1_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(net1_equivalent_regular)

In [ ]:
print_graph_statistics(net2_equivalent_random)
print('\n')
print_statistics_for_largest_component(net2_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(net2_equivalent_regular)

In [ ]:
print_graph_statistics(net3_equivalent_random)
print('\n')
print_statistics_for_largest_component(net3_equivalent_random)
print('\n'*4)
print_statistics_for_largest_component(net3_equivalent_regular)

In [ ]:
get_degree_count_distribution(net1)

In [ ]:
get_degree_count_distribution(net1_equivalent_random)

In [ ]:
get_degree_count_distribution(net1_equivalent_regular)

In [ ]:
get_degree_count_distribution(net2)

In [ ]:
get_degree_count_distribution(net2_equivalent_random)

In [ ]:
get_degree_count_distribution(net2_equivalent_regular)

In [ ]:
get_degree_count_distribution(net3)

In [ ]:
get_degree_count_distribution(net3_equivalent_random)

In [ ]:
get_degree_count_distribution(net3_equivalent_regular)

In [ ]:
print_node_level_comparison(net1, net1_equivalent_random, net1_equivalent_regular)

In [ ]:
print_node_level_comparison(net2, net2_equivalent_random, net2_equivalent_regular)

In [ ]:
print_node_level_comparison(net3, net3_equivalent_random, net3_equivalent_regular)